In [2]:
import json
import gzip
import jsonlines
import yaml
from cdapython import Q, columns, unique_terms, query
Q.set_host_url("http://35.192.60.10:8080")
from collections import defaultdict

In [ ]:
Q("File.dbgap_accession_number is not null")
Q("ResearchSubject.primary_disease_type = 'Lung%' and sex = 'male'")

In [ ]:
# WHERE
#    (((UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage I'))
#        OR (UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage II')))
#      AND (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('Kidney'))
subjects_data_file = 'all_Subjects_3_0.jsonl.gz'
returned_data = []
subjects = []
subject_files = []
researchsubjects = []
rs_files = []
specimens = []
specimen_files = []
diagnoses = []

pass_all_filters = False
with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        pass_all_filters = False
        subj_pass_filters = False
        for RS in subject.get('ResearchSubject'):
            pass_all_filters = False
            RS_filter = RS.get('primary_diagnosis_site')
            if RS_filter is not None and  RS_filter.upper() == 'Kidney'.upper() and RS.get('Diagnosis') is not None:
                for Diagnosis in RS.get('Diagnosis'):
                    if Diagnosis.get('id') == '5197ce2d-2c37-11ec-b712-0a4e2186f121':
                        print(Diagnosis)
                    diag_stage = Diagnosis.get('stage')
                    if diag_stage is not None and diag_stage.upper() in ['Stage I'.upper(), 'Stage II'.upper()]:
                        pass_all_filters = True
                        subj_pass_filters = True
                        #print(Diagnosis)
                        diagnoses.append(Diagnosis)
                        #
            if pass_all_filters and RS.get('Specimen',[]) is not None and RS.get('Specimen',[]) !=[]:
                RS_specimens = RS.pop('Specimen')
                specimens.extend(RS_specimens)
                for spec in RS_specimens:
                    specimen_files.extend(spec.get('Files',[]))
            if pass_all_filters:
                researchsubjects.append(RS)
                rs_files.extend(RS['Files'])
        if subj_pass_filters:
            #subject.pop('ResearchSubject')
            subjects.append(subject)
            subject_files.extend(subject['Files'])


{'id': '5197ce2d-2c37-11ec-b712-0a4e2186f121', 'identifier': [{'value': '5197ce2d-2c37-11ec-b712-0a4e2186f121', 'system': 'PDC'}], 'age_at_diagnosis': '24122', 'primary_diagnosis': 'Renal cell carcinoma, NOS', 'grade': 'G2', 'stage': 'Stage I', 'morphology': '8312/3', 'method_of_diagnosis': 'Not Reported'}


In [8]:
# Remove duplicate Specimens - Cannot have duplicate RS or Subjects
specimens_copy = []
spec_seen = []
for specimen in specimens:
    if specimen['id'] not in spec_seen:
        specimens_copy.append(specimen)
        spec_seen.append(specimen['id'])
subject_files = list(set(subject_files))
rs_files = list(set(rs_files))
specimen_files = list(set(specimen_files))

In [9]:
print("num subjects: " + str(len(subjects)))
print("num subject files: " + str(len(subject_files)))
print("num researchsubjects: " + str(len(researchsubjects)))
print("num researchsubject files: " + str(len(rs_files)))
print("num specimens: " + str(len(specimens_copy)))
print("num specimen files: " + str(len(specimen_files)))
print("num diagnoses: " + str(len(diagnoses)))

sub_ids = []
for subject in subjects:
    sub_ids.append(subject['id'])
rs_ids = []
for rs in researchsubjects:
    rs_ids.append(rs['id'])

num subjects: 65
num subject files: 45342
num researchsubjects: 65
num researchsubject files: 3446
num specimens: 214
num specimen files: 3446
num diagnoses: 65


In [10]:
q1 = Q('ResearchSubject.Diagnosis.stage = "Stage I"')
q2 = Q('ResearchSubject.Diagnosis.stage = "Stage II"')
q3 = Q("ResearchSubject.primary_diagnosis_site = 'Kidney'")
q_diag = q1.Or(q2)
q = q_diag.And(q3)

print("Subject")
qsub = q.subject.count.run()
print(qsub)

print("ResearchSubject")
qrsub = q.researchsubject.count.run()
print(qrsub)
print("Specimen")
qspec = q.specimen.count.run()
print(qspec)
print("Diagnosis")
qdiag = q.diagnosis.count.run()
print(qdiag)


Subject


Getting results from database

Total execution time: 15923 ms


total: 65

files: 45342

...


Diagnosis


Getting results from database

Total execution time: 7608 ms


total: 65

system,count
PDC,65
primary_diagnosis,count
"Renal cell carcinoma, NOS",65
stage,count
Stage I,52
Stage II,13
grade,count
G3,21
G2,38
G1,6


In [11]:
qsub = q.subject.run()
print("Subjects: " + str(len(qsub)))
qrsub = q.researchsubject.run()
print("ResearchSubjects: " + str(len(qrsub)))
qdiag = q.diagnosis.run()
print("Diagnoses: " + str(len(qdiag)))
qtreat = q.treatment.run()
print("Treatments: " + str(len(qtreat)))
qspec = []
for page in q.specimen.run().paginator():
    qspec.extend(page)
print("Specimens: " + str(len(qspec)))

Getting results from database

Total execution time: 3776 ms
Subjects: 65


Getting results from database

Total execution time: 3591 ms
ResearchSubjects: 65


Getting results from database

Total execution time: 3620 ms
Diagnoses: 65


Getting results from database

Total execution time: 3322 ms
Treatments: 0


Getting results from database

Total execution time: 3734 ms
Specimens: 214


In [17]:
blah = data_query_validation('GDC_file_endpoint_mapping.yml')

In [13]:
blah.two_matching_queries(subjects, qsub, 'Patient')

C3L-01288
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00492
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-00799
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-00448
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-00792
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-00800
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-00447
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-01261
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00315
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-00360
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00168
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00495
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00380
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-01836
{'species': {'data': 'homo sapiens', 'que

In [14]:
blah.two_matching_queries(researchsubjects, qrs, 'ResearchSubject')

NameError: name 'qrs' is not defined

In [18]:
blah.two_matching_queries(diagnoses, qdiag, 'Diagnosis')

id: 52481eb1-2c37-11ec-b712-0a4e2186f121
age_at_diagnosis: 18748
<class 'str'>
52481eb1-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '18748', 'query': 18748}}
id: 51fbda73-2c37-11ec-b712-0a4e2186f121
age_at_diagnosis: 15137
<class 'str'>
51fbda73-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '15137', 'query': 15137}}
id: 4fd68b30-2c37-11ec-b712-0a4e2186f121
age_at_diagnosis: 25898
<class 'str'>
4fd68b30-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '25898', 'query': 25898}}
id: 4ec3dd66-2c37-11ec-b712-0a4e2186f121
age_at_diagnosis: 20237
<class 'str'>
4ec3dd66-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '20237', 'query': 20237}}
id: 4f76c1ea-2c37-11ec-b712-0a4e2186f121
age_at_diagnosis: 25512
<class 'str'>
4f76c1ea-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '25512', 'query': 25512}}
id: 51540518-2c37-11ec-b712-0a4e2186f121
age_at_diagnosis: 20938
<class 'str'>
51540518-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'d

In [19]:
print(diagnoses[0]['age_at_diagnosis'])
print(type(diagnoses[0]['age_at_diagnosis']))

24122
<class 'str'>


In [282]:
blah.two_matching_queries(specimens, qspec, 'Specimen')

122d192d-204d-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-17215', 'query': -17215}}
fc4fb74c-2053-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-22179', 'query': -22179}}
a8451484-2053-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-20237', 'query': -20237}}
9c7750a9-2053-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-25345', 'query': -25345}}
6925bdc4-2053-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-26395', 'query': -26395}}
16e97175-2054-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-20974', 'query': -20974}}
bb1f3990-204c-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-32872', 'query': -32872}}
603229bf-204d-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-27092', 'query': -27092}}
15d31398-2054-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-20974', 'query': -20974}}
e554c1bc-2053-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-23908', 'query': -23908}}
da2cf998-2053-11e9-b7f8-0a80fada099c
{'a

In [272]:
qsubf = q.researchsubject.file.run()
print(qsubf)

Getting results from database

Total execution time: 9300 ms

            QueryID: 51cc4a29-e87e-414a-8033-3c4030876024
            
            Offset: 0
            Count: 100
            Total Row Count: 9982
            More pages: True
            


In [274]:
qrs_files=[]
qrs_files_dict = defaultdict(list)
for page in q.researchsubject.file.run().paginator():
    qrs_files.extend(page)
    for file in page:
        qrs_files_dict[file['id']].append(file)
print(len(qrs_files))
print(len(qrs_files_dict))

Getting results from database

Total execution time: 3945 ms
9982
3446


In [276]:
qrs_files_distinct = []
for file_id, rec_lst in qrs_files_dict.items():
    qrs_files_distinct.append(rec_lst[0])

In [286]:
#rs_files is list of ids, not records. 
files_data_file = 'all_Files_3_0.jsonl.gz'
rs_files_full = []
with gzip.open(files_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for file in reader:
        if file['id'] in rs_files:
            rs_files_full.append(file)

In [287]:
blah.two_matching_queries(rs_files_full, qrs_files_distinct, 'File')

783c6368-1b4c-11e9-8e2e-005056921935
{'byte_size': {'data': '177374614', 'query': 177374614}}
5881172c-1b50-11e9-8e2e-005056921935
{'byte_size': {'data': '186714564', 'query': 186714564}}
fd51b576-1b56-11e9-9ac1-005056921935
{'byte_size': {'data': '6100435', 'query': 6100435}}
ea68f83c-1b53-11e9-a7d8-005056921935
{'byte_size': {'data': '4122300', 'query': 4122300}}
36195c44-1b46-11e9-b79b-005056921935
{'byte_size': {'data': '178168375', 'query': 178168375}}
256ddb31-f1f8-11e9-9a07-0a80fada099c
{'byte_size': {'data': '1715344392', 'query': 1715344392}}
37d285d0-1a04-11e9-b898-005056921935
{'byte_size': {'data': '631952436', 'query': 631952436}}
30c19046-1b4f-11e9-8e2e-005056921935
{'byte_size': {'data': '169007972', 'query': 169007972}}
8d6e4080-19d5-11e9-99db-005056921935
{'byte_size': {'data': '510765986', 'query': 510765986}}
dfd87c3a-1b53-11e9-a7d8-005056921935
{'byte_size': {'data': '3919820', 'query': 3919820}}
fbbbb17a-1b57-11e9-9ac1-005056921935
{'byte_size': {'data': '7581961',

{'byte_size': {'data': '2230310', 'query': 2230310}}
01ab76f2-1b4d-11e9-8e2e-005056921935
{'byte_size': {'data': '250712013', 'query': 250712013}}
6e583a80-1b46-11e9-b79b-005056921935
{'byte_size': {'data': '186921271', 'query': 186921271}}
b6dd9a30-1b54-11e9-b4de-005056921935
{'byte_size': {'data': '2965894', 'query': 2965894}}
1a826846-1a05-11e9-b898-005056921935
{'byte_size': {'data': '757254864', 'query': 757254864}}
c15a7ff4-1b55-11e9-b4de-005056921935
{'byte_size': {'data': '2303612', 'query': 2303612}}
ff802ca8-19d3-11e9-99db-005056921935
{'byte_size': {'data': '713474234', 'query': 713474234}}
1483964e-1b55-11e9-b4de-005056921935
{'byte_size': {'data': '3140783', 'query': 3140783}}
d38b4098-1b53-11e9-a7d8-005056921935
{'byte_size': {'data': '4071727', 'query': 4071727}}
da2a937e-1b4c-11e9-8e2e-005056921935
{'byte_size': {'data': '190674980', 'query': 190674980}}
dc4e3418-1b41-11e9-b79b-005056921935
{'byte_size': {'data': '207546389', 'query': 207546389}}
3060d6d2-19d0-11e9-99db

400aa38a-1b4f-11e9-8e2e-005056921935
{'byte_size': {'data': '187566173', 'query': 187566173}}
094a9332-1b54-11e9-a7d8-005056921935
{'byte_size': {'data': '3885887', 'query': 3885887}}
e7a1d640-19c9-11e9-99db-005056921935
{'byte_size': {'data': '689908752', 'query': 689908752}}
c0f60af8-1b53-11e9-a7d8-005056921935
{'byte_size': {'data': '3769888', 'query': 3769888}}
534be2aa-1b55-11e9-b4de-005056921935
{'byte_size': {'data': '1616956', 'query': 1616956}}
9fef7406-19d3-11e9-99db-005056921935
{'byte_size': {'data': '703795466', 'query': 703795466}}
256fb2b6-f1f8-11e9-9a07-0a80fada099c
{'byte_size': {'data': '2001004096', 'query': 2001004096}}
5d2ff9fa-19d4-11e9-99db-005056921935
{'byte_size': {'data': '770776350', 'query': 770776350}}
c4193d86-19cd-11e9-99db-005056921935
{'byte_size': {'data': '848034500', 'query': 848034500}}
cdd2a04c-1b58-11e9-b01e-005056921935
{'byte_size': {'data': '2990612', 'query': 2990612}}
fd618684-1b58-11e9-b01e-005056921935
{'byte_size': {'data': '5650846', 'qu

3c8a6608-19d0-11e9-99db-005056921935
{'byte_size': {'data': '737657428', 'query': 737657428}}
c0245780-1b56-11e9-9ac1-005056921935
{'byte_size': {'data': '6029931', 'query': 6029931}}
c5b72f18-1a04-11e9-b898-005056921935
{'byte_size': {'data': '642775146', 'query': 642775146}}
61fb4f24-1b56-11e9-9ac1-005056921935
{'byte_size': {'data': '7641308', 'query': 7641308}}
c503d760-1b58-11e9-b01e-005056921935
{'byte_size': {'data': '2190847', 'query': 2190847}}
75be7352-19ca-11e9-99db-005056921935
{'byte_size': {'data': '763859568', 'query': 763859568}}
50486736-1b55-11e9-b4de-005056921935
{'byte_size': {'data': '3108495', 'query': 3108495}}
0c1477be-1b58-11e9-9ac1-005056921935
{'byte_size': {'data': '6440347', 'query': 6440347}}
20fa3812-1b58-11e9-9ac1-005056921935
{'byte_size': {'data': '7176581', 'query': 7176581}}
482881c6-1b55-11e9-b4de-005056921935
{'byte_size': {'data': '2532492', 'query': 2532492}}
2b7e6810-1b46-11e9-b79b-005056921935
{'byte_size': {'data': '269453492', 'query': 269453

{'byte_size': {'data': '208252813', 'query': 208252813}}
c3b98378-1b53-11e9-a7d8-005056921935
{'byte_size': {'data': '4838911', 'query': 4838911}}
2124e7bc-19d0-11e9-99db-005056921935
{'byte_size': {'data': '801632226', 'query': 801632226}}
5036b166-1a02-11e9-b898-005056921935
{'byte_size': {'data': '735277328', 'query': 735277328}}
68ef87d2-1b56-11e9-9ac1-005056921935
{'byte_size': {'data': '6701543', 'query': 6701543}}
78cf5d70-19cc-11e9-99db-005056921935
{'byte_size': {'data': '744310974', 'query': 744310974}}
fb61e5b2-1a00-11e9-b898-005056921935
{'byte_size': {'data': '714314148', 'query': 714314148}}
fdd1b3f6-1b57-11e9-9ac1-005056921935
{'byte_size': {'data': '7281911', 'query': 7281911}}
0f305254-1b55-11e9-b4de-005056921935
{'byte_size': {'data': '3350597', 'query': 3350597}}
265a1ccc-1b54-11e9-a7d8-005056921935
{'byte_size': {'data': '3024178', 'query': 3024178}}
021f301c-1b50-11e9-8e2e-005056921935
{'byte_size': {'data': '160717529', 'query': 160717529}}
7b06f6ea-1b55-11e9-b4de

{'byte_size': {'data': '6786349', 'query': 6786349}}
331784bc-1b46-11e9-b79b-005056921935
{'byte_size': {'data': '263678125', 'query': 263678125}}
4fd870d2-1b57-11e9-9ac1-005056921935
{'byte_size': {'data': '7459586', 'query': 7459586}}
725bc5f0-1b43-11e9-b79b-005056921935
{'byte_size': {'data': '224521252', 'query': 224521252}}
7aca4e48-19ca-11e9-99db-005056921935
{'byte_size': {'data': '728732340', 'query': 728732340}}
d0ba1d62-1b53-11e9-a7d8-005056921935
{'byte_size': {'data': '1612434', 'query': 1612434}}
7634d682-1b46-11e9-b79b-005056921935
{'byte_size': {'data': '188643277', 'query': 188643277}}
508c723c-1b50-11e9-8e2e-005056921935
{'byte_size': {'data': '194926521', 'query': 194926521}}
7ab48e14-19d4-11e9-99db-005056921935
{'byte_size': {'data': '733754876', 'query': 733754876}}
b387866e-1b42-11e9-b79b-005056921935
{'byte_size': {'data': '193727826', 'query': 193727826}}
b86ce2a4-1b57-11e9-9ac1-005056921935
{'byte_size': {'data': '7812004', 'query': 7812004}}
fa0413c8-1b56-11e9-

e966418a-1b56-11e9-9ac1-005056921935
{'byte_size': {'data': '6762218', 'query': 6762218}}
c1c4b2ea-1b58-11e9-b01e-005056921935
{'byte_size': {'data': '2805367', 'query': 2805367}}
0c26b6ae-19d2-11e9-99db-005056921935
{'byte_size': {'data': '911735672', 'query': 911735672}}
1054a47e-1b54-11e9-a7d8-005056921935
{'byte_size': {'data': '5123740', 'query': 5123740}}
b951374a-1b54-11e9-b4de-005056921935
{'byte_size': {'data': '2819036', 'query': 2819036}}
9f48d67a-1b57-11e9-9ac1-005056921935
{'byte_size': {'data': '7307575', 'query': 7307575}}
2e740d04-1b46-11e9-b79b-005056921935
{'byte_size': {'data': '276033527', 'query': 276033527}}
2f698d2a-1b54-11e9-a7d8-005056921935
{'byte_size': {'data': '3464468', 'query': 3464468}}
a72e919e-1a04-11e9-b898-005056921935
{'byte_size': {'data': '708667190', 'query': 708667190}}
df7311ce-1b58-11e9-b01e-005056921935
{'byte_size': {'data': '4250849', 'query': 4250849}}
efddedde-19ca-11e9-99db-005056921935
{'byte_size': {'data': '774798704', 'query': 774798

c8b3a61e-1a05-11e9-b898-005056921935
{'byte_size': {'data': '720818510', 'query': 720818510}}
bc3a056e-1b58-11e9-b01e-005056921935
{'byte_size': {'data': '5630264', 'query': 5630264}}
49d26212-1b55-11e9-b4de-005056921935
{'byte_size': {'data': '2564052', 'query': 2564052}}
383af57e-1a00-11e9-b898-005056921935
{'byte_size': {'data': '756277430', 'query': 756277430}}
02e5150e-1b4e-11e9-8e2e-005056921935
{'byte_size': {'data': '152247688', 'query': 152247688}}
144d5c74-1b54-11e9-a7d8-005056921935
{'byte_size': {'data': '4168152', 'query': 4168152}}
39c7c9f8-1b54-11e9-a7d8-005056921935
{'byte_size': {'data': '3412558', 'query': 3412558}}
5c7d6880-1b55-11e9-b4de-005056921935
{'byte_size': {'data': '3192409', 'query': 3192409}}
256e3384-f1f8-11e9-9a07-0a80fada099c
{'byte_size': {'data': '1467017544', 'query': 1467017544}}
88c4432e-19d3-11e9-99db-005056921935
{'byte_size': {'data': '715258824', 'query': 715258824}}
e379e77a-1b53-11e9-a7d8-005056921935
{'byte_size': {'data': '3649360', 'query'

fd22c89a-1b58-11e9-b01e-005056921935
{'byte_size': {'data': '4925674', 'query': 4925674}}
b8252dba-1b57-11e9-9ac1-005056921935
{'byte_size': {'data': '7676243', 'query': 7676243}}
b8ad7a0e-1b4c-11e9-8e2e-005056921935
{'byte_size': {'data': '188893652', 'query': 188893652}}
44fa2b94-1b55-11e9-b4de-005056921935
{'byte_size': {'data': '3429135', 'query': 3429135}}
cd850a48-19d3-11e9-99db-005056921935
{'byte_size': {'data': '724119756', 'query': 724119756}}
16e2943c-1b58-11e9-9ac1-005056921935
{'byte_size': {'data': '6811343', 'query': 6811343}}
256e03a3-f1f8-11e9-9a07-0a80fada099c
{'byte_size': {'data': '1340503364', 'query': 1340503364}}
c8409d08-1b41-11e9-b79b-005056921935
{'byte_size': {'data': '219857937', 'query': 219857937}}
20e39b88-1a00-11e9-b898-005056921935
{'byte_size': {'data': '711192594', 'query': 711192594}}
b746e09e-1b54-11e9-b4de-005056921935
{'byte_size': {'data': '2401448', 'query': 2401448}}
37929338-1a01-11e9-b898-005056921935
{'byte_size': {'data': '771550476', 'quer

726a1b6e-1b57-11e9-9ac1-005056921935
{'byte_size': {'data': '6856236', 'query': 6856236}}
c332634a-1b4c-11e9-8e2e-005056921935
{'byte_size': {'data': '161777722', 'query': 161777722}}
90da824e-19d3-11e9-99db-005056921935
{'byte_size': {'data': '700249938', 'query': 700249938}}
49cf604c-1b43-11e9-b79b-005056921935
{'byte_size': {'data': '190238636', 'query': 190238636}}
8f64c23c-19cc-11e9-99db-005056921935
{'byte_size': {'data': '753732570', 'query': 753732570}}
d7985988-19d1-11e9-99db-005056921935
{'byte_size': {'data': '976811116', 'query': 976811116}}
fb498a3c-1b57-11e9-9ac1-005056921935
{'byte_size': {'data': '8057743', 'query': 8057743}}
bb0071dc-1b54-11e9-b4de-005056921935
{'byte_size': {'data': '1840270', 'query': 1840270}}
be4d89b6-1b44-11e9-b79b-005056921935
{'byte_size': {'data': '200444140', 'query': 200444140}}
d6e1679a-1b58-11e9-b01e-005056921935
{'byte_size': {'data': '1806800', 'query': 1806800}}
ecc6f322-1b53-11e9-a7d8-005056921935
{'byte_size': {'data': '4311519', 'quer

In [22]:
sub_id = 'C3L-00902'
break_all = False
with gzip.open('all_Subjects_3_0.jsonl.gz', 'r') as gdc:
    reader = jsonlines.Reader(gdc)
    for i in reader:
        if i['id'] == sub_id:
            for rs in i['ResearchSubject']:
                if rs['identifier'][0]['system'] == 'GDC':
                    print(rs['Diagnosis'])
                    print(type(rs['Diagnosis'][0]['age_at_diagnosis']))
                    break_all= True
                    break
        if break_all:
            break

[{'id': '82bc10e0-19a9-4fbf-a0cd-007b98c96660', 'identifier': [{'value': '82bc10e0-19a9-4fbf-a0cd-007b98c96660', 'system': 'GDC'}], 'primary_diagnosis': 'Renal cell carcinoma, NOS', 'age_at_diagnosis': 25512, 'morphology': '8312/3', 'stage': None, 'grade': 'G1', 'method_of_diagnosis': None, 'Treatment': []}]
<class 'int'>


In [30]:

break_all = False
with gzip.open('all_Subjects_3_0.jsonl.gz', 'r') as gdc:
    reader = jsonlines.Reader(gdc)
    for i in reader:
        for rs in i.get('ResearchSubject'):
            if rs['identifier'][0]['system'] == 'GDC':
                for d in rs.get('Diagnosis'):
                    print(d)
                    print(d.get('age_at_diagnosis'))
                break_all= True
                break
        if break_all:
            break

{'id': '82bc10e0-19a9-4fbf-a0cd-007b98c96660', 'identifier': [{'value': '82bc10e0-19a9-4fbf-a0cd-007b98c96660', 'system': 'GDC'}], 'primary_diagnosis': 'Renal cell carcinoma, NOS', 'age_at_diagnosis': 25512, 'morphology': '8312/3', 'stage': None, 'grade': 'G1', 'method_of_diagnosis': None, 'Treatment': []}
25512


In [ ]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "brain"')
results = myquery.specimen.count.run()
results

In [ ]:
#Usage test query 1 from ReadTheDocs
#q1 = Q('ResearchSubject.Diagnosis.Treatment.treatment_type = "Radiation Therapy, NOS"')
#q2 = Q('ResearchSubject.identifier.system = "PDC"')
#q3 = Q('ResearchSubject.identifier.system = "GDC"')
#q = q2.From(q1.And(q3))
# Above query requires GDC Diagnosis Treatment type to be Radiation Therapy, NOS, but also have PDC information 
# At least that is how the query gets written. Funny enough PDC does not populate Treatment entities so another
# filter requiring both PDC and GDC to have that info would give zero results.
# Donovan's edit: I'm not sure yet. Keep as is?
subjects = []
subject_files = []
researchsubjects = []
rs_files = []
specimens = []
specimen_files = []

with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        pass_all_filters = False
        subj_pass_filters = False
        has_GDC_RS = False
        has_PDC_RS = False
        for RS in subject['ResearchSubject']:
            RS_filter = RS['identifier']['system']
            if RS_filter == 'GDC' and RS.get('Diagnosis') is not None:
                for Diagnosis in RS.get('Diagnosis'):
                    if Diagnosis.get('Treatment') is not None:
                        for Treatment in Diagnosis.get('Treatment'):
                            pass_all_filters = True
                            subj_pass_filters = True
                            break
            if pass_all_filters and RS.get('Specimen',[]) is not None and RS.get('Specimen',[]) !=[]:
                RS_specimens = RS.pop('Specimen')
                specimens.extend(RS_specimens)
                for spec in RS_specimens:
                    specimen_files.extend(spec.get('Files',[]))
            if pass_all_filters:
                researchsubjects.append(RS)
                rs_files.extend(RS['Files'])
        if subj_pass_filters:
            #subject.pop('ResearchSubject')
            subjects.append(subject)
            subject_files.extend(subject['Files'])



In [ ]:
# Usage test query 2
#q1 = Q('ResearchSubject.associated_project = "TCGA-BRCA"')
#q2 = Q('days_to_birth > -50*365')
#q3 = Q('identifier.system = "IDC"')
#q = q3.From(q1.And(q2))
# Subject over 50, has an RS with project "TCGA-BRCA" ugh... IDC does not list project as TCGA-BRCA, but tcga_brca
# I really need to be more vigilant with testing and writing these queries
# Donovan's edit: change associated project to tcga_brca
subjects = []
subject_files = []
researchsubjects = []
rs_files = []
specimens = []
specimen_files = []

with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        pass_all_filters = False
        subj_pass_filters = False
        #print(subject.get('days_to_birth', 0.0))
        if subject.get('days_to_birth', 0.0) is not None and int(subject.get('days_to_birth', 0.0)) < -50*365:
            for RS in subject['ResearchSubject']:
                RS_filter = RS.get('member_of_research_project')
                if RS_filter is not None and  RS_filter.upper() == 'tcga_brca'.upper():
                    pass_all_filters = True
                    subj_pass_filters = True
                if pass_all_filters and RS.get('Specimen',[]) is not None and RS.get('Specimen',[]) !=[]:
                    RS_specimens = RS.pop('Specimen')
                    specimens.extend(RS_specimens)
                    for spec in RS_specimens:
                        specimen_files.extend(spec.get('Files',[]))
                if pass_all_filters:
                    researchsubjects.append(RS)
                    rs_files.extend(RS['Files'])
        if subj_pass_filters:
            #subject.pop('ResearchSubject')
            subjects.append(subject)
            subject_files.extend(subject['Files'])



In [ ]:
# Remove duplicate Specimens - Cannot have duplicate RS or Subjects
specimens_copy = []
spec_seen = []
ind = []
for specimen in specimens:
    if specimen['id'] not in spec_seen:
        specimens_copy.append(specimen)
subject_files = list(set(subject_files))
rs_files = list(set(rs_files))
specimen_files = list(set(specimen_files))

In [ ]:
print(len(subjects))
print(len(researchsubjects))
print(len(specimens_copy))
print(len(subject_files))
print(len(rs_files))
print(len(specimen_files))

In [ ]:
#Find data from patients with lung cancer under 45 years, 
# males with no smoking exposure with Radiomic (CT, PET images) data, mass spec data, WES or WGS
# Donovan's edit: We do not currently have exposures in our data model. 
#                 We can try to do the rest of the filters though.
q1 = Q('days_to_birth > -50*365')

qsub = q1.subjects()
print(qsub)
print(q1.to_json())

In [ ]:
#FInd RNA seq and proteomics data for patients under age 16 from tissue samples of the brain from CPTAC program.


In [ ]:
#Find data for patients from TCGA-COAD with a Diagnosis of stage iii or greater and morphology 8140/3 across ethnicities.

In [ ]:
subjects_data_file = 'all_Subjects_3_0_w_RS.jsonl.gz'
sample_case_mapping = defaultdict(list)
#WHERE
#    (((UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage I'))
#        OR (UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage II')))
#      AND (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('Kidney'))
pass_all_filters = False
all_subjects_specimens = []
with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        for RS in subject['ResearchSubject']:
            for spec in RS.get('Specimen',[]):
                if subject['id'] in sub_ids:
                    all_subjects_specimens.append(spec)
                sample_case_mapping[spec['id']].append(RS['id'])


In [ ]:
rs_specs = []
for spec,cases in sample_case_mapping.items():
    if len(cases) > 1:
        print(sample_case_mapping[spec])
    if cases[0] in rs_ids:
        rs_specs.append(spec)

In [ ]:
with gzip.open('IDC_V8_Subjects_Full.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    with gzip.open('IDC_V8_Subjects_Full_v2.jsonl.gz', 'w') as subj:
        writer = jsonlines.Writer(subj)
        for subject in reader:
            for RS in subject['ResearchSubject']:
                RS['File'] = subject['File']
            writer.write(subject)

In [ ]:
with gzip.open('IDC_V8_Subjects_Full_v2.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        print(subject['ResearchSubject'])
        break

In [ ]:
with gzip.open('gdc.all_cases.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        for spec in subject['samples']:
            print(spec)
            break
        break

In [ ]:
with gzip.open('data/pdc.files-specimens-cases.json.gz', "r") as f_in:
    files_per_sample_dict = json.load(f_in)
specimen_w_out_files = {}
for spec in files_per_sample_dict:
    if files_per_sample_dict[spec] is None:
        specimen_w_out_files[spec] = files_per_sample_dict[spec]
print(len(specimen_w_out_files))

In [ ]:
#files_per_sample_dict['3b96e558-b060-48ec-a1d5-98554c53afc2']
files_per_sample_dict['cecdcea4-7b7e-40fb-a013-e4db87604f67']

In [ ]:
with open('gdc.all_cases.H.jsonl', 'r') as subj:
    for subject in subj:
        line = json.loads(subject)
        print(line)
        break

In [ ]:
import json

In [ ]:
type(line)

In [ ]:
lst = ['a', 'b', 'c']
for i,j in enumerate(lst):
    print(i)

In [ ]:
blah = {'a':1,'b':2}
while len(blah)>0:
    list(blah.keys())[0]
    blah.pop(list(blah.keys())[0])
blah

In [ ]:
#blah.pop('b')
blah
len(blah)

In [ ]:
count = 0
seen_subjects = {}
with gzip.open('test_merge_v2.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjects:
            seen_subjects[subject['id']] += 1
        else:
            seen_subjects[subject['id']] = 1
print(count)

In [ ]:
for key, val in seen_subjects.items():
    if val > 1:
        print(key)

In [ ]:
q1 = Q('ResearchSubject.primary_diagnosis_site = "ovary"')
q1.subject.run()

In [ ]:
q1.research_subject.run()

In [ ]:
q1.specimen.run(limit=1000)

In [ ]:
specs = q1.specimen.run(limit=1000)

In [ ]:
spec_ids = []
for page in specs.paginator():
    for spec in page:
        if spec['id'] not in spec_ids:
            spec_ids.append(spec['id'])
        else:
            print(spec)

In [ ]:
print(len(spec_ids))
print(len(set(spec_ids)))
print(len(spec_seen))

In [ ]:
subjects_data_file = 'all_Subjects_3_0_w_RS.jsonl.gz'
returned_data = []
subjects = []
subject_files = []
researchsubjects = []
rs_files = []
specimens = []
specimen_files = []
diagnoses = []
with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        pass_all_filters = False
        subj_pass_filters = False
        for RS in subject['ResearchSubject']:
            pass_all_filters = False
            RS_filter = RS.get('primary_diagnosis_site')
            if RS_filter is not None and  RS_filter.upper() == 'ovary'.upper():
                pass_all_filters = True
                subj_pass_filters = True
                researchsubjects.append(RS)
                specimens.extend(RS.get('Specimen',[]))
                diagnoses.extend(RS.get('Diagnosis',[]))
        if subj_pass_filters:
            subject.pop('ResearchSubject')
            subjects.append(subject)

In [ ]:
specimens_copy = []
spec_seen = []
for specimen in specimens:
    if specimen['id'] not in spec_seen:
        specimens_copy.append(specimen)
        spec_seen.append(specimen['id'])
subject_files = list(set(subject_files))
rs_files = list(set(rs_files))
specimen_files = list(set(specimen_files))

In [ ]:
print(len(specimens))

In [ ]:
print(len(subjects))
print(len(researchsubjects))
print(len(diagnoses))
print(len(specimens_copy))
print(len(subject_files))
print(len(rs_files))
print(len(specimen_files))
sub_ids = []
for subject in subjects:
    sub_ids.append(subject['id'])
rs_ids = []
for rs in researchsubjects:
    rs_ids.append(rs['id'])

In [ ]:
diff = spec_ids==spec_seen
diff

In [ ]:
spec_seen[0]

In [ ]:
res = [x for x in spec_seen + spec_ids if x not in spec_seen or x not in spec_ids]
res

In [ ]:
if None in spec_ids:
    print('why?!')

In [ ]:
spec_ids = []
for page in specs.paginator():
    for spec in page:
        if spec['id'] is None:
            print(spec)

In [ ]:
q1 = Q('ResearchSubject.primary_diagnosis_site = "ovary"')
q2 = Q('ResearchSubject.identifier.system = "IDC"')
q = q1.And(q2)
q.research_subject.run()

In [ ]:
q.specimen.run()

In [ ]:
q_uh_oh = q.specimen.run()
q_uh_oh[0]

In [ ]:
qidcrs = q.research_subject.run()
qidcrs[0]

In [ ]:
count = 0
seen_subjects = {}
with gzip.open('IDC_V8_Subjects_Full_v2.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjects:
            seen_subjects[subject['id']] += 1
        else:
            seen_subjects[subject['id']] = 1
print(count)
print(len(seen_subjects))

In [ ]:
count = 0
seen_subjectsgdc = {}
with gzip.open('gdc.all_cases.A.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjectsgdc:
            seen_subjectsgdc[subject['id']] += 1
        else:
            seen_subjectsgdc[subject['id']] = 1
print(count)
print(len(seen_subjectsgdc))

In [ ]:
count = 0
seen_subjectspdc = {}
with gzip.open('pdc.all_cases.A.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjectspdc:
            seen_subjectspdc[subject['id']] += 1
        else:
            seen_subjectspdc[subject['id']] = 1
print(count)
print(len(seen_subjectspdc))

In [ ]:
all_sub = []
all_sub.extend(seen_subjects)
all_sub.extend(seen_subjectsgdc)
all_sub.extend(seen_subjectspdc)
all_sub = set(all_sub)
len(all_sub)

In [ ]:
for idd, count in seen_subjects.items():
    if count>1:
        print(idd)
        break

In [ ]:
count = 0
seen_subjectsidc = {}
with gzip.open('IDC_V8_Subjects_Full.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjectsidc:
            seen_subjectsidc[subject['id']] += 1
        else:
            seen_subjectsidc[subject['id']] = 1
print(count)
print(len(seen_subjectsidc))

In [ ]:
count = 0
seen_subjectsidc_ = {}
with gzip.open('idc_v8_Subjects_w_RS.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjectsidc_:
            seen_subjectsidc_[subject['id']] += 1
        else:
            seen_subjectsidc_[subject['id']] = 1
print(count)
print(len(seen_subjectsidc_))

In [ ]:
for i in seen_subjectsidc_:
    print(i)
    break

In [ ]:
count = 0
seen_subjectsidc = {}
with gzip.open('idc_v8_Subjects.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    with gzip.open('idc_v8_Subjects_v2.jsonl.gz', 'w') as subj_out:
        writer = jsonlines.Writer(subj_out)
        for subject in reader:
            count+=1
            if subject['id'] in seen_subjectsidc:
                seen_subjectsidc[subject['id']] += 1
            else:
                seen_subjectsidc[subject['id']] = 1
            for RS in subject['ResearchSubject']:
                RS['File'] = subject['File']
            writer.write(subject)
print(count)
print(len(seen_subjectsidc))

In [ ]:
count = 0
seen_subjectsidc = {}
max_bit_size = 104857600
import sys
with gzip.open('../../Downloads/public_R3_gcp-merge2.jsonl (2).gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    break_all = False
    for subject in reader:
        line_size = sys.getsizeof(subject) + sys.getsizeof(subject['File'])
        count += 1
        for rsubject in subject['ResearchSubject']:
            line_size += sys.getsizeof(rsubject) + sys.getsizeof(rsubject['File'])
            if "Specimen" in rsubject:
                for spec in rsubject['Specimen']:
                    line_size += sys.getsizeof(spec) + sys.getsizeof(spec['File'])
            if "Diagnosis" in rsubject:
                for d in rsubject['Diagnosis']:
                    line_size+=sys.getsizeof(d)
        if count in [95823, 95824, 95825]:
            print(subject['identifier'])
            print('num files: ' + str(len(subject['File'])))
            print(sys.getsizeof(subject))
            print(sys.getsizeof(subject['File']))
            print(line_size)
        if count > 95825:
            break

In [ ]:
def create_udf_str(func_desc):
    var_list = ['x','y','z','a','b','c']
    out = "CREATE TEMP FUNCTION "+ func_desc[0] + "("
    for var in range(len(func_desc[1])):
        out += (var_list[var] + " " + func_desc[1][var])
        if var == len(func_desc[1])-1:
            out + ") RETURNS "
        else:
            out += ", "
    out = out + func_desc[2][0] + " AS ("
    if len(func_desc[1]) == 1:
        out = out + func_desc[0]()
    else:
        out = out + func_desc[0](func_desc[1][1:])
    out = out + "); "
    return out
func = ['name', ['string','string', 'string'], 'string']
create_udf_str(func)

In [ ]:
data_file = 'all_Files_3_0.jsonl.gz'
with gzip.open(data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    gdc = open('gdc_drs_uri.txt', 'w')
    pdc = open('pdc_drs_uri.txt', 'w')
    idc = open('idc_drs_uri.txt', 'w')
    for subject in reader:
        if subject['identifier'][0]['system'] == 'GDC':
            gdc.write(subject['drs_uri'])
            gdc.write( '\n')
        elif subject['identifier'][0]['system'] == 'PDC':
            pdc.write(subject['drs_uri'])
            pdc.write( '\n')
        elif subject['identifier'][0]['system'] == 'IDC':
            idc.write(subject['drs_uri'])
            idc.write( '\n')

In [ ]:
red = ['a','b','c']
with open('drs_uri.txt', 'w') as out:
    for subject in red:
        out.write(subject)
        out.write( '\n')

In [ ]:
subjects_data_file = 'gdc.all_cases.jsonl.gz'
break_all = False
with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        if subject['case_id'] == 'fef9c64f-5959-4da0-aaa2-66b56fc7b4c3':
            for samp in subject['samples']:
                for portion in samp['portions']:
                    for analyte in portion['analytes']:
                        if analyte['analyte_id'] =='e7224386-1bc6-4e96-afe5-7cac425ed3a9':
                            print(portion['portion_id'])
                            print(analyte)

In [23]:
from cdatransform.transform.validate import LogValidation
import logging

In [46]:
logger = logging.getLogger('data_explorer')
logging.basicConfig(
        filename='data_explorer.txt',
        format="%(asctime)s %(levelname)-8s %(message)s",
        level=logging.INFO,
    )

validate = LogValidation()
validate.agree_sources({'sub1': subjects[0], 'sub1.1': subjects[50]}, 
                       subjects[0]['id'],
                       ['species', 'sex', 'race', 'ethnicity', 'days_to_birth',
                        'vital_status',
                        'age_at_death',
                        'cause_of_death',])
validate.generate_report(logger)

In [45]:
subjects[50]

{'id': 'C3L-00800',
 'identifier': [{'value': 'C3L-00800', 'system': 'GDC'},
  {'value': 'C3L-00800', 'system': 'PDC'},
  {'system': 'IDC', 'value': 'C3L-00800'}],
 'race': 'white',
 'ethnicity': 'not reported',
 'sex': 'male',
 'days_to_birth': -15279,
 'species': 'homo sapiens',
 'subject_associated_project': ['cptac_ccrcc', 'CPTAC3-Discovery', 'CPTAC-3'],
 'vital_status': 'Alive',
 'age_at_death': None,
 'cause_of_death': 'Not Reported',
 'ResearchSubject': [{'id': '1256a2fe-44c4-4019-a5bc-d188e3c76079',
   'identifier': [{'value': '1256a2fe-44c4-4019-a5bc-d188e3c76079',
     'system': 'GDC'}],
   'member_of_research_project': 'CPTAC-3',
   'primary_diagnosis_condition': 'Adenomas and Adenocarcinomas',
   'primary_diagnosis_site': 'Kidney',
   'Files': ['0beccacb-bc6e-4118-af83-5c807c3aae8f',
    'a76c2ddd-ecaa-4a98-b837-379eb73b3189',
    '75735ce3-da3e-41e3-afbc-6f9724a050bb',
    'a44e9101-b014-4ba3-8840-1b2e5feb5be2',
    '331a18f4-69cc-40ba-a580-013de2742cd7',
    'fb7796ef-6cd

In [16]:
import yaml
from yaml import Loader
from collections import defaultdict
class data_query_validation:
    def __init__(self, mapping_yaml) -> None:
        # map of field name -> set of values
        self.mapping_yaml = mapping_yaml
        self.entity_fields = self._init_entity_fields()
    def _init_entity_fields(self):
        return_dict = {}
        MandT = yaml.load(open(self.mapping_yaml, "r"), Loader=Loader)
        for entity, MorT_dict in MandT.items():
            return_dict[entity.replace('_merge','')] = MorT_dict['Mapping']
        return return_dict
    def two_matching_recs(self, recs, entity_name):
        diffs = {}
        same = True
        rec_sources = list(recs.keys())
        for field in self.entity_fields[entity_name]:
            rec_compare = [recs[source].get(field) for source in rec_sources]
            #type_of_compare = ''
            for i in rec_compare:
                if isinstance(rec_compare[0],list) and isinstance(rec_compare[1],list):
                    if not isinstance(rec_compare[0][0],dict):
                        rec_compare[0].sort()
                        rec_compare[1].sort()
                if rec_compare[0] != rec_compare[1]:
                    if None not in rec_compare:
                        same = False
                        diffs[field] = {rec_sources[0]:rec_compare[0],
                                        rec_sources[1]:rec_compare[1]}
        return same, diffs
    def all_identical_dups(self, results_dict, entity_name):
        # determines if all duplicate entities are identical
        all_identical = True
        all_diffs = defaultdict(list)
        for idd, recs in results_dict.items():
            for i in range(1, len(recs)):
                same, diffs = self.two_matching_recs({0:recs[0], i:recs[i]}, entity_name)
                if not same:
                    all_identical = False
                    all_diffs[idd].append(diffs)
                else:
                    all_diffs[idd].append(None)
        return all_identical, all_diffs
                    
            
    def two_matching_queries(self, data_file_results, query_results, entity):
        df_ids = set([i['id'] for i in data_file_results])
        qr_ids = set([i['id'] for i in query_results])
        return_same_ids = len(df_ids) == len(qr_ids)
        return_same_num_recs = len(data_file_results)==len(query_results)
        dups_in_data_return = len(df_ids) == len(data_file_results)
        dups_in_query_return = len(qr_ids) == len(query_results)
        no_dups = dups_in_data_return and dups_in_query_return
        missing_ids = []
        added_ids = []
        ids_w_diff_data = []
        #Simple comparison - same ids and number of entities returned
        if return_same_num_recs and return_same_ids and no_dups:
            df_recs = {}
            qr_recs = {}
            for idd in df_ids:
                for rec in data_file_results:
                    if rec['id'] == idd:
                        df_recs[idd] = rec
                        print('id: ' + idd)
                        print('age_at_diagnosis: ' + str(rec['age_at_diagnosis']))
                        print(type(rec['age_at_diagnosis']))
                        break
                for rec in query_results:
                    if rec['id'] == idd:
                        qr_recs[idd] = rec
                        break
                        
                match, diffs = self.two_matching_recs({'data': df_recs[idd], 
                                                       'query': qr_recs[idd]}, entity)
                if not match:
                    print(idd)
                    print(diffs)
        #elif len(data_file_results) != len(query_results) and df_ids == qr_ids:
            # There must be duplicates if distinct ids are the same.
            # Reduce dups in longer file - check that data is the same for the dups
            # If not then need to report that error as well.
        else:
            print('fucked')
            

In [252]:
blah = data_query_validation('GDC_subject_endpoint_mapping.yml')

In [240]:
same, diffs = blah.two_matching_recs({'data':subjects[0], 'query': subjects[1]}, 'Patient')
same
diffs

{'id': {'data': 'C3N-00494', 'query': 'C3N-00148'},
 'identifier': {'data': [{'value': 'C3N-00494', 'system': 'GDC'},
   {'value': 'C3N-00494', 'system': 'PDC'},
   {'system': 'IDC', 'value': 'C3N-00494'}],
  'query': [{'value': 'C3N-00148', 'system': 'GDC'},
   {'value': 'C3N-00148', 'system': 'PDC'},
   {'system': 'IDC', 'value': 'C3N-00148'}]},
 'days_to_birth': {'data': -24122, 'query': -19314}}

In [261]:
blah.two_matching_queries(subjects, qsub, 'Patient')

C3L-00902
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00953
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-01882
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00317
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00168
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00242
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-00917
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-00796
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00148
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-00154
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3N-01522
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-00010
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-01283
{'species': {'data': 'homo sapiens', 'query': 'Homo sapiens'}}
C3L-01603
{'species': {'data': 'homo sapiens', 'que

In [260]:
q1 = Q('ResearchSubject.Diagnosis.stage = "Stage I"')
q2 = Q('ResearchSubject.Diagnosis.stage = "Stage II"')
q3 = Q("ResearchSubject.primary_diagnosis_site = 'Kidney'")
q_diag = q1.Or(q2)
q = q_diag.And(q3)

print("Subject")
qsub = q.subject.run()

Subject


Getting results from database

Total execution time: 4086 ms


In [242]:
qrs = q.researchsubject.run()

Getting results from database

Total execution time: 5957 ms


In [243]:
blah.two_matching_queries(researchsubjects, qrs, 'ResearchSubject')

In [254]:
qspec = q.specimen.run()

Getting results from database

Total execution time: 4191 ms


In [255]:
qspec = []
for page in q.specimen.run().paginator():
    qspec.extend(page)
blah.two_matching_queries(specimens, qspec, 'Specimen')
print(len(specimens))
print(len(qspec))


Getting results from database

Total execution time: 4224 ms
122d192d-204d-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-17215', 'query': -17215}}
fc4fb74c-2053-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-22179', 'query': -22179}}
a8451484-2053-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-20237', 'query': -20237}}
9c7750a9-2053-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-25345', 'query': -25345}}
6925bdc4-2053-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-26395', 'query': -26395}}
16e97175-2054-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-20974', 'query': -20974}}
bb1f3990-204c-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-32872', 'query': -32872}}
603229bf-204d-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-27092', 'query': -27092}}
15d31398-2054-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-20974', 'query': -20974}}
e554c1bc-2053-11e9-b7f8-0a80fada099c
{'age_at_collection': {'data': '-23908', 'query': -23908}}
da2cf998-2

In [262]:
qdiag = q.diagnosis.run()

Getting results from database

Total execution time: 4881 ms


In [263]:
blah.two_matching_queries(diagnoses, qdiag, 'Diagnosis')

4f1f6c24-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '16335', 'query': 16335}}
4f00e0cc-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '22351', 'query': 22351}}
4eb49c02-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '19213', 'query': 19213}}
4f84b909-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '22179', 'query': 22179}}
5065c7a8-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '17820', 'query': 17820}}
50ac2c7b-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '25345', 'query': 25345}}
5107b0dc-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '11619', 'query': 11619}}
5076079e-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '19314', 'query': 19314}}
4fa8b95d-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '13526', 'query': 13526}}
5116f037-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '28865', 'query': 28865}}
51ce2f4d-2c37-11ec-b712-0a4e2186f121
{'age_at_diagnosis': {'data': '28

In [205]:
s1 = {'a': [{'id': 'a', 'sex': 'male', 'race': 'w', 'subject_associated_project': ['q','r']},
            {'id': 'a', 'sex': 'male', 'race': 'w', 'subject_associated_project': ['q','r']},
            {'id': 'a', 'sex': 'female', 'race': 'w', 'subject_associated_project': ['q','s']}, 
            {'id': 'a', 'sex': 'Male', 'race': 'b', 'subject_associated_project': ['q','r']}],
     'b': [{'id': 'a', 'sex': 'female', 'race': 'w', 'subject_associated_project': ['q','r']},
            {'id': 'a', 'sex': 'female', 'race': 'w', 'subject_associated_project': ['q','r']},
            {'id': 'a', 'sex': 'female', 'race': 'w', 'subject_associated_project': ['q','r']}, 
            {'id': 'a', 'sex': 'Male', 'race': 'b', 'subject_associated_project': ['q','r']}]}
blah.all_identical_dups(s1, 'Patient')


(False,
 defaultdict(list,
             {'a': [None,
               {'sex': ['male', 'female'],
                'subject_associated_project': [['q', 'r'], ['q', 's']]},
               {'sex': ['male', 'Male'], 'race': ['w', 'b']}],
              'b': [None,
               None,
               {'sex': ['female', 'Male'], 'race': ['w', 'b']}]}))

In [202]:
s1 = {'a': [{'id': 'a', 'sex': 'male', 'race': 'w'},
            {'id': 'a', 'sex': 'male', 'race': 'w'},
            {'id': 'a', 'sex': 'male', 'race': 'w'},
            {'id': 'a', 'sex': 'male', 'race': 'w'}]}
blah.all_identical_dups(s1, 'Patient')

(True, defaultdict(list, {'a': [None, None, None]}))

In [217]:
q1 = Q('ResearchSubject.Diagnosis.stage = "Stage I"')
q2 = Q('ResearchSubject.Diagnosis.stage = "Stage II"')
q3 = Q("ResearchSubject.primary_diagnosis_site = 'Kidney'")
q_diag = q1.Or(q2)
q = q_diag.And(q3)

print("Subject")
qsub = q3.file.run()

Subject


Getting results from database

Total execution time: 17598 ms


In [218]:
qsub[0]

{'id': '91017612-67fb-4858-9e2a-4002a60d493d',
 'identifier': [{'system': 'GDC',
   'value': '91017612-67fb-4858-9e2a-4002a60d493d'}],
 'label': 'nationwidechildrens.org_biospecimen_protocol_read.txt',
 'data_category': 'Biospecimen',
 'data_type': 'Biospecimen Supplement',
 'file_format': 'BCR Biotab',
 'associated_project': 'TCGA-READ',
 'drs_uri': 'drs://dg.4DFC:91017612-67fb-4858-9e2a-4002a60d493d',
 'byte_size': '149940',
 'checksum': '60e9f9253e5322eb35492b2f01b7de65',
 'data_modality': 'Genomic',
 'imaging_modality': None,
 'dbgap_accession_number': None,
 'researchsubject_specimen_id': '',
 'researchsubject_id': '',
 'subject_id': 'TCGA-AG-A002'}

In [219]:
query_dict = defaultdict(list)
for i in qsub:
    query_dict[i['id']].append(i)

In [223]:
len(query_dict)==len(qsub)
print(len(query_dict))
print(len(qsub))

97
100


In [222]:
blahf = data_query_validation('GDC_file_endpoint_mapping.yml')
blahf.all_identical_dups(query_dict, 'File')

(True,
 defaultdict(list,
             {'ce1b271c-1a5a-450e-97a6-b223f5a6b4a8': [None],
              '91faef29-f263-4ba8-bbdc-0243a8353870': [None, None]}))

In [224]:
query_dict['ce1b271c-1a5a-450e-97a6-b223f5a6b4a8']

[{'id': 'ce1b271c-1a5a-450e-97a6-b223f5a6b4a8',
  'identifier': [{'system': 'GDC',
    'value': 'ce1b271c-1a5a-450e-97a6-b223f5a6b4a8'}],
  'label': 'nationwidechildrens.org_clinical_follow_up_v1.0_coad.txt',
  'data_category': 'Clinical',
  'data_type': 'Clinical Supplement',
  'file_format': 'BCR Biotab',
  'associated_project': 'TCGA-COAD',
  'drs_uri': 'drs://dg.4DFC:ce1b271c-1a5a-450e-97a6-b223f5a6b4a8',
  'byte_size': '142645',
  'checksum': 'caf7631b355a496a00e5931117491721',
  'data_modality': 'Genomic',
  'imaging_modality': None,
  'dbgap_accession_number': None,
  'researchsubject_specimen_id': '',
  'researchsubject_id': '',
  'subject_id': 'TCGA-AA-A01I'},
 {'id': 'ce1b271c-1a5a-450e-97a6-b223f5a6b4a8',
  'identifier': [{'system': 'GDC',
    'value': 'ce1b271c-1a5a-450e-97a6-b223f5a6b4a8'}],
  'label': 'nationwidechildrens.org_clinical_follow_up_v1.0_coad.txt',
  'data_category': 'Clinical',
  'data_type': 'Clinical Supplement',
  'file_format': 'BCR Biotab',
  'associated